# Dynamic Window Approach Simulation 
<div class="alert alert-block alert-info">
<b>Explanation:</b> In robotics motion planning, the dynamic window approach (DWA) is an online collision avoidance strategy for mobile robots. 
DWA allows a robot to dynamically adjust its path in real time, taking into account its capabilities and the environment around it. 
By focusing on short-term movements rather than a complete path, DWA is effective for navigating complex environments while avoiding obstacles.
</div>

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import math
from Planner.local_planner import dwa_control, motion, plot_robot, plot_arrow
from IPython.display import clear_output, display, Markdown

ModuleNotFoundError: No module named 'Planner'

## Loading the Obstacles
<div class="alert alert-block alert-info">
<b>Explanation:</b> 

- The code below converts the map data into a list of obstacles, each representing an obstacle's position (x, y) and a fixed radius for plotting.

</div>

In [162]:

# Load obstacle data from file
with open('map.txt', 'r') as file:
    map_data = json.load(file)

# Define constants
SIM_LOOP = 500
radius = 0.07
area = 1.5  # Set area size to center the plot on a larger grid

# Convert obstacle data to a list of obstacles with fixed radius
obstacle_list = [np.array((data['x'], data['y'], radius)) for data in map_data.values()]

## Simulation Parameters
<div class="alert alert-block alert-info">
<b>Explanation:</b> 


The parameters that need to be set include:
- **Maximum Speed (max_speed)** 
    - The maximum speed that the vehicle can travel.
    
    
    
- **Minimum Speed (min_speed)** 
    - The minimum speed that the vehicle can travel.
- **Maximum Acceleration (max_accel)** 
    - The maximum acceleration of the vehicle.  
- **Maximum Angular velocity (max_yaw_rate)** 
    - The maximum angular velocity of the vehicle. 
- **Predicted time (predict_time)**  
    - The predicted/estimated time to reach the goal without hitting any obstacles. 


</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> The angular velocity is the rate of change of the position angle of an object with respect to time. 
</div>

<div class="alert alert-block alert-danger">
<b>Your Task:</b> Set the values of the five parameters described above; the goal is to achieve the lowest number of iterations in the simulation without any crashes. The person with the fewest iterations will win a chocolate!
</div>

In [163]:

class Config:
    """
    simulation parameter class
    """

    def __init__(self, obs_arr):
        # robot parameter
        # PARAMETERS THAT NEED TO BE SET (Replace _ with actual values):
        # ======================================== To Do ============================================== #
        self.max_speed = _  # [m/s]
        self.min_speed = _  # [m/s]
        self.max_yaw_rate = _  # [rad/s]
        self.max_accel = _  # [m/ss]
        self.predict_time = _ # [s]
        # ============================================================================================= # 

        # Parameters that are kept constant. 
        self.max_delta_yaw_rate = 40.0 * math.pi / 180.0  # [rad/ss]
        self.v_resolution = 0.01  # [m/s]
        self.yaw_rate_resolution = 0.1 * math.pi / 180.0  # [rad/s]
        self.dt = 0.1  # [s] Time tick for motion prediction
        self.to_goal_cost_gain = 0.15
        self.speed_cost_gain = 1
        self.obstacle_cost_gain = 0.05
        self.robot_stuck_flag_cons = 0.001  # constant to prevent robot stucked

        # if robot_type == RobotType.circle
        # Also used to check if goal is reached in both types
        self.robot_radius = 0.05  # [m] for collision check
        self.ob = obs_arr

config = Config(obstacle_list)


## Simulation Loop 
<div class="alert alert-block alert-info">
<b>Explanation:</b> 

- Plots each iteration in a loop to create a simulation. 

- The plot includes:
    - <span style="color:blue">Solid blue circle</span> to show the current location of vehicle.
    - A <span style="color:black">black arrow</span> to indicate the direction of the vehicle 
    - A <span style="color:green">green line</span> to show the predicted trajectory.
    - <span style="color:red">Red circles</span> are the obstacles 
    - The <span style="color:blue">blue cross</span> is the goal i.e. where the vehicle must reach 
</div>

<div class="alert alert-block alert-warning">
<b>Note:</b> The number of iterations is shown as the title of the plot. 
</div>

In [ ]:


# Initial state of the robot [x, y, yaw, v, omega]
x = np.array([0.0, 0.0, math.pi / 8.0, 0.0, 0.0])

# Goal position
goal = np.array([1, 1])
show_animation = True

# Main simulation loop
for i in range(SIM_LOOP):


    print(f'Iteration: {i}')
    
    # Clear plot for each iteration
    clear_output(wait=True)
    
    plt.cla()
    # Check for ESC key to stop simulation
    plt.gcf().canvas.mpl_connect('key_release_event', lambda event: [exit(0) if event.key == 'escape' else None])

    # Plot obstacles
    for obstacle in obstacle_list:
        ob_x, ob_y, ob_r = obstacle
        circle = plt.Circle((ob_x, ob_y), radius=ob_r, color='r', fill=False)
        plt.gca().add_patch(circle)

    # Run DWA control to get input (u) and predicted trajectory
    ob = np.array([[obs[0], obs[1]] for obs in obstacle_list])  # Convert obstacles to 2D list for DWA
    u, predicted_trajectory = dwa_control(x, config, goal, ob)

    # Update robot state based on motion model
    x = motion(x, u, config.dt)

    # Plot the predicted trajectory and robot position
    if show_animation:
        plt.plot(predicted_trajectory[:, 0], predicted_trajectory[:, 1], "-g")  # DWA predicted trajectory
        plt.plot(x[0], x[1], "xr")  # Current position of robot
        plt.plot(goal[0], goal[1], "xb")  # Goal position
        plot_robot(x[0], x[1], x[2], config)  # Plot the robot
        plot_arrow(x[0], x[1], x[2])  # Plot direction arrow

    # Set plot limits and grid
    plt.xlim(-area, area)
    plt.ylim(-area, area)
    plt.grid(True)
    plt.title(f"Iteration {i + 1}")
    plt.gca().set_aspect('equal', adjustable='box')

    # Check if goal is reached
    dist_to_goal = math.hypot(x[0] - goal[0], x[1] - goal[1])
    if dist_to_goal <= config.robot_radius:  # Assume robot radius or goal tolerance of 0.2 meters
        print("Goal reached!")
        break
    # Pause to simulate real-time update and display
    plt.pause(0.01)


print("Simulation finished.")

